In [48]:
#Libraries
import numpy as np
import pandas as pd

#Connection to DB
from sqlalchemy import create_engine
driver = 'mysql+pymysql'
ip = 'barcelona-db.cyxhqbnhiohl.eu-west-3.rds.amazonaws.com'
username = 'admin'
password = 'PercyA2019!'
db = 'project2'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'
# Engine & Query
engine = create_engine(connection_string)

In [49]:
#Step 1 - Load dataframe
stg_df = pd.read_csv("2018_accidents_vehicles_gu_bcn_.csv")    

In [50]:
#Step 2 - Check Data Types
stg_df.dtypes

Codi_expedient               object
Codi_districte                int64
Nom_districte                object
Codi_barri                    int64
Nom_barri                    object
Codi_carrer                   int64
Nom_carrer                   object
Num_postal                   object
Descripcio_dia_setmana       object
Dia_setmana                  object
Descripcio_tipus_dia         object
Any                           int64
Mes_any                       int64
Nom_mes                      object
Dia_mes                       int64
Hora_dia                      int64
Descripcio_torn              object
Descripcio_causa_vianant     object
Descripcio_tipus_vehicle     object
Descripcio_model             object
Descripcio_marca             object
Descripcio_color             object
Descripcio_carnet            object
Antiguitat_carnet            object
Coordenada_UTM_X            float64
Coordenada_UTM_Y            float64
Longitud                    float64
Latitud                     

In [51]:
#Step 3 - Rename Problem Column Name
stg_df.rename(columns={'Num_postal ':'Num_postal'}, inplace=True)

In [52]:
#Step 4 - Remove whitespace in selected columns
stg_df['Codi_expedient_clean'] = stg_df.loc[:, 'Codi_expedient'].str.strip()
stg_df['Nom_carrer_clean'] = stg_df.loc[:, 'Nom_carrer'].str.strip()

In [56]:
# Step X - Create a 'datetime column'
stg_df['Date'] = stg_df[['Dia_mes', 'Mes_any', 'Any']].apply(lambda x: str(x.Dia_mes)+ '-' + str(x.Mes_any) + '-' + str(x.Any), axis=1)
stg_df['Date'] = stg_df.Date.astype('datetime64[ns]')

In [57]:
#Step X - Create 4 new data frames from the raw file
accident = stg_df[['Codi_expedient_clean','Codi_carrer','Descripcio_causa_vianant']]
accident_datetime = stg_df[['Codi_expedient_clean','Hora_dia','Dia_mes','Mes_any','Any', 'Date']]
accident_location = stg_df[['Codi_expedient_clean','Coordenada_UTM_X','Coordenada_UTM_Y','Longitud','Latitud']]
accident_street = stg_df[['Codi_expedient_clean','Nom_carrer_clean','Nom_barri','Nom_districte']]

In [59]:
#Step X - Push new DFs to SQL
stg_df.to_sql('stg_data', con=engine, if_exists='replace')
accident.to_sql('stg_accident', con=engine, if_exists='replace')
accident_datetime.to_sql('stg_accident_datetime', con=engine, if_exists='replace')
accident_location.to_sql('stg_accident_location', con=engine, if_exists='replace')
accident_street.to_sql('stg_street', con=engine, if_exists='replace')